In [81]:
import yaml
from pprint import pprint
import dicttoxml
# there is also dict2xml available in pypy (python3)
import xml.etree.ElementTree as ET


In [82]:
with open('../data/zynqmon.yml') as f:
    y = yaml.load(f, Loader=yaml.FullLoader)
    pprint(y)


{'cm_hw_rev': '1A',
 'config': [{'count': 25,
             'mcu_call': 'firefly_temps',
             'name': 'firefly',
             'names': ['K01  12 Tx GTH',
                       'K01  12 Rx GTH',
                       'K02  12 Tx GTH',
                       'K02  12 Rx GTH',
                       'K03  12 Tx GTH',
                       'K03  12 Rx GTH',
                       'K04 4 XCVR GTY',
                       'K05 4 XCVR GTY',
                       'K06 4 XCVR GTY',
                       'K07  12 Tx GTY',
                       'K07  12 Rx GTY',
                       'V01 4 XCVR GTY',
                       'V02 4 XCVR GTY',
                       'V03 4 XCVR GTY',
                       'V04 4 XCVR GTY',
                       'V05 4 XCVR GTY',
                       'V06 4 XCVR GTY',
                       'V07 4 XCVR GTY',
                       'V08 4 XCVR GTY',
                       'V09 4 XCVR GTY',
                       'V10 4 XCVR GTY',
                   

In [83]:
# This is the parent (root) tag
# onto which other tags would be
# created
data = ET.Element('node')
data.set('id', 'PL_MEM')


In [84]:
cm = ET.SubElement(data, 'node')
cm.set('id', 'CM')
cm.set('addr', '0x00000000')


In [85]:
config = y['config']

for c in config:
    print(c)
    start = c['start']
    count = c['count']
    i = 0
    names = c['names']
    for n in names:
        print(n)
        node = ET.SubElement(cm, 'node')
        node.set('id', n.replace(' ', '_'))
        addr = int((start + i)/2)
        remain = (start + i) % 2
        node.set('addr', str(hex(addr)))
        if ( remain == 1 ):
            node.set('mask', "0x000000FF")
        else:
            node.set('mask', "0x00FF0000")
        i += 1
        # this appears to be on all the nodes
        node.set("permission", "r")
        # set parameter based on the type
        if ( c['type'] == 'int8' ):
            node.set("format", "d")
        elif ( c['type'] == 'fp16' ):
            node.set("format", "f")
        elif ( c['type'] == 'char' ):
            node.set("format", "c")
        else:
            print("ERROR: unknown type", c['type'])
            break
        #print(ET.tostring(node))


{'name': 'firefly', 'start': 0, 'count': 25, 'mcu_call': 'firefly_temps', 'type': 'int8', 'names': ['K01  12 Tx GTH', 'K01  12 Rx GTH', 'K02  12 Tx GTH', 'K02  12 Rx GTH', 'K03  12 Tx GTH', 'K03  12 Rx GTH', 'K04 4 XCVR GTY', 'K05 4 XCVR GTY', 'K06 4 XCVR GTY', 'K07  12 Tx GTY', 'K07  12 Rx GTY', 'V01 4 XCVR GTY', 'V02 4 XCVR GTY', 'V03 4 XCVR GTY', 'V04 4 XCVR GTY', 'V05 4 XCVR GTY', 'V06 4 XCVR GTY', 'V07 4 XCVR GTY', 'V08 4 XCVR GTY', 'V09 4 XCVR GTY', 'V10 4 XCVR GTY', 'V11  12 Tx GTY', 'V11  12 Rx GTY', 'V12  12 Tx GTY', 'V12  12 Rx GTY']}
K01  12 Tx GTH
K01  12 Rx GTH
K02  12 Tx GTH
K02  12 Rx GTH
K03  12 Tx GTH
K03  12 Rx GTH
K04 4 XCVR GTY
K05 4 XCVR GTY
K06 4 XCVR GTY
K07  12 Tx GTY
K07  12 Rx GTY
V01 4 XCVR GTY
V02 4 XCVR GTY
V03 4 XCVR GTY
V04 4 XCVR GTY
V05 4 XCVR GTY
V06 4 XCVR GTY
V07 4 XCVR GTY
V08 4 XCVR GTY
V09 4 XCVR GTY
V10 4 XCVR GTY
V11  12 Tx GTY
V11  12 Rx GTY
V12  12 Tx GTY
V12  12 Rx GTY
{'name': 'psmon', 'start': 32, 'count': 8, 'mcu_call': 'psmon', 'type': 'f

In [88]:
s_xml = ET.tostring(data)


In [89]:
from bs4 import BeautifulSoup

bs = BeautifulSoup(s_xml, 'xml')
pretty_xml = bs.prettify()
print(pretty_xml)


<?xml version="1.0" encoding="utf-8"?>
<node id="PL_MEM">
 <node addr="0x00000000" id="CM">
  <node addr="0x0" format="d" id="K01__12_Tx_GTH" mask="0x00FF0000" permission="r"/>
  <node addr="0x0" format="d" id="K01__12_Rx_GTH" mask="0x000000FF" permission="r"/>
  <node addr="0x1" format="d" id="K02__12_Tx_GTH" mask="0x00FF0000" permission="r"/>
  <node addr="0x1" format="d" id="K02__12_Rx_GTH" mask="0x000000FF" permission="r"/>
  <node addr="0x2" format="d" id="K03__12_Tx_GTH" mask="0x00FF0000" permission="r"/>
  <node addr="0x2" format="d" id="K03__12_Rx_GTH" mask="0x000000FF" permission="r"/>
  <node addr="0x3" format="d" id="K04_4_XCVR_GTY" mask="0x00FF0000" permission="r"/>
  <node addr="0x3" format="d" id="K05_4_XCVR_GTY" mask="0x000000FF" permission="r"/>
  <node addr="0x4" format="d" id="K06_4_XCVR_GTY" mask="0x00FF0000" permission="r"/>
  <node addr="0x4" format="d" id="K07__12_Tx_GTY" mask="0x000000FF" permission="r"/>
  <node addr="0x5" format="d" id="K07__12_Rx_GTY" mask="0x